In [2]:
import torch
import transformers

# import
import openai

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import OpenAIEmbedding

from langchain.llms import Ollama
import chromadb

from dotenv import dotenv_values

/Users/kiranhk/anaconda3/envs/pyml-book/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api_key = dotenv_values('.env')["OPENAI_API_KEY"]
openai.api_key = api_key

In [4]:
#Set the embedding

llm = Ollama(model="llama2")
#embed_model = OllamaEmbeddings(base_url="http://localhost:11434", model="llama2") #Local Llama 2 embedding model
embed_model = OpenAIEmbedding() #Using OpenAI's text-embed-002

In [5]:
COLLECTION = "aiprof"
PATH = './chroma'

In [6]:
# create client and a new collection
db = chromadb.PersistentClient(path=PATH)
chroma_collection = db.get_or_create_collection(COLLECTION)

In [7]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kiranhk/Library/Caches/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# load documents
documents = SimpleDirectoryReader("data").load_data()

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

In [ ]:
# load from disk
db2 = chromadb.PersistentClient(path=PATH)
chroma_collection = db2.get_or_create_collection(COLLECTION)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index2 = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [ ]:
query_engine = index2.as_query_engine()

In [ ]:
resp = query_engine.query("What is The rational agent approach?")
print(resp.response)

In [ ]:
resp = query_engine.query("Generate 3 concise questions from the PDFs")


In [ ]:
resp.response.split('\n')